In [6]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from keras.models import load_model 
fpath = '../data/davis/'
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
ligands = json.load(open(fpath+"ligands_can.txt"), object_pairs_hook=OrderedDict)
proteins = json.load(open(fpath+"proteins.txt"), object_pairs_hook=OrderedDict)

Y = pickle.load(open(fpath + "Y","rb"), encoding='latin1') ### TODO: read from raw

In [8]:
perfmeasure = get_cindex
runmethod = build_combined_categorical
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = 32
FLAGS.seq_window_lengths = [4]
FLAGS.smi_window_lengths = [4]
FLAGS.batch_size = 256
FLAGS.num_epoch = 100
FLAGS.max_seq_len = 1000
FLAGS.max_smi_len = 100
FLAGS.dataset_path= '../data/davis/'
FLAGS.problem_type = 1
FLAGS.log_dir = '../logs'


In [9]:
dataset = DataSet( fpath = FLAGS.dataset_path, ### BUNU ARGS DA GUNCELLE
                      setting_no = FLAGS.problem_type, ##BUNU ARGS A EKLE
                      seqlen = FLAGS.max_seq_len,
                      smilen = FLAGS.max_smi_len,
                      need_shuffle = False )
# set character set size
FLAGS.charseqset_size = dataset.charseqset_size 
FLAGS.charsmiset_size = dataset.charsmiset_size 

XD, XT, Y = dataset.parse_data(fpath = FLAGS.dataset_path)

XD = np.asarray(XD)
XT = np.asarray(XT)
Y = np.asarray(Y)

drugcount = XD.shape[0]
print(drugcount)
targetcount = XT.shape[0]
print(targetcount)

FLAGS.drug_count = drugcount
FLAGS.target_count = targetcount

label_row_inds, label_col_inds = np.where(np.isnan(Y)==False)  #basically finds the point address of affinity [x,y]

if not os.path.exists(figdir):
    os.makedirs(figdir)
test_set, outer_train_sets = dataset.read_sets(FLAGS.dataset_path, FLAGS.problem_type) 


Reading ../data/davis/ start
Read ../data/davis/ start
68
442
Reading ../data/davis/ start
val set 5010
train set 20036


In [52]:

logging("---Parameter Search-----", FLAGS)

Y = np.mat(np.copy(Y))

params = {}

XD = XD[label_row_inds]
XT = XT[label_col_inds]

train_drugs, train_prots,  train_Y = prepare_interaction_pairs(XD, XT, Y, label_row_inds, label_col_inds)

In [53]:

data_path='../../data/'
XD_dtc, XT_dtc, Y_dtc = get_DTC_train(data_path+'dtc_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len)


In [72]:
from sklearn.model_selection import train_test_split
all_train_drugs = np.concatenate((np.asarray(train_drugs), np.asarray(XD_dtc)), axis=0)
all_train_prots = np.concatenate((np.asarray(train_prots), np.asarray(XT_dtc)), axis=0)
all_train_Y = np.concatenate((np.asarray(train_Y), np.asarray(Y_dtc)), axis=0)
all_train_Y = -np.log10(1e-8+all_train_Y/1e9)

XD_train, XD_val, XT_train, XT_val, Y_train, Y_val = train_test_split(all_train_drugs, all_train_prots, all_train_Y, 
                                                                      test_size=0.1, 
                                                                     random_state=15)


In [80]:
model_name='checkpoints/davis_dtc_dta'
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)
checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

gridmodel = build_combined_categorical(FLAGS, FLAGS.num_windows, FLAGS.smi_window_lengths[0], FLAGS.seq_window_lengths[0])
gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=FLAGS.batch_size, epochs=1000, 
        validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
           , callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)

gridmodel.save('davis_dtc_dta.h5')

Train on 51395 samples, validate on 5711 samples
Epoch 1/1000
Epoch 00001: val_loss improved from inf to 2.25803, saving model to checkpoints/davis_dtc_dta.h5
 - 46s - loss: 1.2989 - cindex_score: 0.6293 - val_loss: 2.2580 - val_cindex_score: 0.4955
Epoch 2/1000
Epoch 00002: val_loss improved from 2.25803 to 0.76028, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 0.6249 - cindex_score: 0.6759 - val_loss: 0.7603 - val_cindex_score: 0.4783
Epoch 3/1000
Epoch 00003: val_loss did not improve
 - 38s - loss: 0.5492 - cindex_score: 0.6989 - val_loss: 2.3011 - val_cindex_score: 0.5860
Epoch 4/1000
Epoch 00004: val_loss improved from 0.76028 to 0.55174, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 0.5078 - cindex_score: 0.7178 - val_loss: 0.5517 - val_cindex_score: 0.6980
Epoch 5/1000
Epoch 00005: val_loss improved from 0.55174 to 0.51736, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 0.4927 - cindex_score: 0.7262 - val_loss: 0.5174 - val_cindex_score: 

Epoch 51/1000
Epoch 00051: val_loss improved from 0.41095 to 0.40865, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 0.3540 - cindex_score: 0.8062 - val_loss: 0.4086 - val_cindex_score: 0.7738
Epoch 52/1000
Epoch 00052: val_loss did not improve
 - 38s - loss: 0.3557 - cindex_score: 0.8053 - val_loss: 0.4387 - val_cindex_score: 0.7560
Epoch 53/1000
Epoch 00053: val_loss improved from 0.40865 to 0.40716, saving model to checkpoints/davis_dtc_dta.h5
 - 38s - loss: 0.3535 - cindex_score: 0.8046 - val_loss: 0.4072 - val_cindex_score: 0.7819
Epoch 54/1000
Epoch 00054: val_loss did not improve
 - 38s - loss: 0.3531 - cindex_score: 0.8062 - val_loss: 0.4116 - val_cindex_score: 0.7791
Epoch 55/1000
Epoch 00055: val_loss did not improve
 - 38s - loss: 0.3535 - cindex_score: 0.8060 - val_loss: 0.4205 - val_cindex_score: 0.7720
Epoch 56/1000
Epoch 00056: val_loss did not improve
 - 38s - loss: 0.3536 - cindex_score: 0.8069 - val_loss: 0.4173 - val_cindex_score: 0.7725
Epoch 57/1000
Ep

In [ ]:
from sklearn.metrics import mean_squared_error, auc, f1_score
from scipy.stats import pearsonr, spearmanr

gridmodel = load_model('davis_dtc_dta.h5')
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)


In [82]:
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
print('cindex:', get_cindex(Y_val, predicted_labels))
print('rmse:', np.sqrt(mean_squared_error(Y_val, predicted_labels)))
#print('pearsonr:', pearsonr(Y_val, predicted_labels[:, 0]))
#print('spearmanr:', np.sqrt(spearmanr(val_Y, predicted_labels[:, 0])))
print('f1:', np.sqrt(f1_score(Y_val>7, predicted_labels>7)))


cindex: [0.77902714]
rmse: 0.6334978811975447
f1: 0.7442827651721986


In [83]:
XD_test, XT_test = get_DTC_train(data_path+'dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])


In [45]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv(data_path+'submission_file.csv', index=None)